In [114]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
%matplotlib inline

In [115]:
#df = pd.read_csv('byStation.csv')
#df.columns = ['STATION','DATE','ENTRIES','EXITS']
df_final = pd.read_pickle('df_final.pickle')
df_final.head()

,STATION,DATE,ENTRIES,EXITS,name,score,geo
0,1 AV,01/01/2018,8427,9924,1st Ave,11,"(40.6508606878022, -73.94945514035334)"
1,1 AV,01/02/2018,18668,21328,1st Ave,11,"(40.6508606878022, -73.94945514035334)"
2,1 AV,01/03/2018,20417,22357,1st Ave,11,"(40.6508606878022, -73.94945514035334)"
3,1 AV,01/04/2018,12010,13472,1st Ave,11,"(40.6508606878022, -73.94945514035334)"
4,1 AV,01/05/2018,18891,21966,1st Ave,11,"(40.6508606878022, -73.94945514035334)"


In [116]:
df = df_final.drop(['score'],axis=1)
df['weekday'] = pd.to_datetime(df['DATE']).dt.weekday
df.head()

,STATION,DATE,ENTRIES,EXITS,name,geo,weekday
0,1 AV,01/01/2018,8427,9924,1st Ave,"(40.6508606878022, -73.94945514035334)",0
1,1 AV,01/02/2018,18668,21328,1st Ave,"(40.6508606878022, -73.94945514035334)",1
2,1 AV,01/03/2018,20417,22357,1st Ave,"(40.6508606878022, -73.94945514035334)",2
3,1 AV,01/04/2018,12010,13472,1st Ave,"(40.6508606878022, -73.94945514035334)",3
4,1 AV,01/05/2018,18891,21966,1st Ave,"(40.6508606878022, -73.94945514035334)",4


In [117]:
df[df.STATION=='WORLD TRADE CTR'].head()
### NOT SAME as station data: World Trade Center POINT (-74.00974461517701 40.71256392680817)

,STATION,DATE,ENTRIES,EXITS,name,geo,weekday
135072,WORLD TRADE CTR,01/01/2018,6332,6918,World Trade Center,"(40.61925870977273, -73.99884094850685)",0
135073,WORLD TRADE CTR,01/02/2018,17020,13749,World Trade Center,"(40.61925870977273, -73.99884094850685)",1
135074,WORLD TRADE CTR,01/03/2018,17349,13917,World Trade Center,"(40.61925870977273, -73.99884094850685)",2
135075,WORLD TRADE CTR,01/04/2018,9140,7387,World Trade Center,"(40.61925870977273, -73.99884094850685)",3
135076,WORLD TRADE CTR,01/05/2018,14394,12494,World Trade Center,"(40.61925870977273, -73.99884094850685)",4



URL	OBJECTID	NAME	the_geom	LINE	NOTES
408	http://web.mta.info/nyct/service/	409	World Trade Center	POINT (-74.00974461517701 40.71256392680817)	E	E-all times

In [118]:
sta_map = df.groupby('STATION').agg(lambda x: list(x)[0])
sta_map.head()

,DATE,ENTRIES,EXITS,name,geo,weekday
STATION,,,,,,
1 AV,01/01/2018,8427,9924,1st Ave,"(40.6508606878022, -73.94945514035334)",0
103 ST,01/01/2018,11233,8290,103rd St,"(40.76008683231326, -73.9752485052734)",0
103 ST-CORONA,01/01/2018,6809,5523,103rd St,"(40.76008683231326, -73.9752485052734)",0
104 ST,01/01/2018,746,243,104th St,"(40.84386300128381, -73.83632199755944)",0
110 ST,01/01/2018,3986,3582,110th St,"(40.617397744443736, -73.9592431052215)",0


In [119]:
df_sta = pd.read_pickle('station_loc.pickle')
df_sta = df_sta.set_index('NAME')
df_sta = df_sta[~df_sta.index.duplicated(keep='first')]
df_sta['lat'] = df_sta.the_geom.apply(lambda s: float(s[7:-1].split(' ')[1]))
df_sta['long'] = df_sta.the_geom.apply(lambda s: float(s[7:-1].split(' ')[0]))
df_sta.head()

,URL,OBJECTID,the_geom,LINE,NOTES,lat,long
NAME,,,,,,,
Astor Pl,http://web.mta.info/nyct/service/,1,POINT (-73.99106999861966 40.73005400028978),4-6-6 Express,"4 nights, 6-all times, 6 Express-weekdays AM s...",40.730054,-73.991070
Canal St,http://web.mta.info/nyct/service/,2,POINT (-74.00019299927328 40.71880300107709),4-6-6 Express,"4 nights, 6-all times, 6 Express-weekdays AM s...",40.718803,-74.000193
50th St,http://web.mta.info/nyct/service/,3,POINT (-73.98384899986625 40.76172799961419),1-2,"1-all times, 2-nights",40.761728,-73.983849
Bergen St,http://web.mta.info/nyct/service/,4,POINT (-73.97499915116808 40.68086213682956),2-3-4,"4-nights, 3-all other times, 2-all times",40.680862,-73.974999
Pennsylvania Ave,http://web.mta.info/nyct/service/,5,POINT (-73.89488591154061 40.66471445143568),3-4,"4-nights, 3-all other times",40.664714,-73.894886


In [120]:
df['lat'] = df.name.apply(lambda sta: df_sta.loc[sta].lat)
df['long'] = df.name.apply(lambda sta: df_sta.loc[sta].long)
df.head()

,STATION,DATE,ENTRIES,EXITS,name,geo,weekday,lat,long
0,1 AV,01/01/2018,8427,9924,1st Ave,"(40.6508606878022, -73.94945514035334)",0,40.730975,-73.981681
1,1 AV,01/02/2018,18668,21328,1st Ave,"(40.6508606878022, -73.94945514035334)",1,40.730975,-73.981681
2,1 AV,01/03/2018,20417,22357,1st Ave,"(40.6508606878022, -73.94945514035334)",2,40.730975,-73.981681
3,1 AV,01/04/2018,12010,13472,1st Ave,"(40.6508606878022, -73.94945514035334)",3,40.730975,-73.981681
4,1 AV,01/05/2018,18891,21966,1st Ave,"(40.6508606878022, -73.94945514035334)",4,40.730975,-73.981681


In [121]:
df['volume'] = df.ENTRIES + df.EXITS
df = df.drop('geo',axis=1)
df.head()

,STATION,DATE,ENTRIES,EXITS,name,weekday,lat,long,volume
0,1 AV,01/01/2018,8427,9924,1st Ave,0,40.730975,-73.981681,18351
1,1 AV,01/02/2018,18668,21328,1st Ave,1,40.730975,-73.981681,39996
2,1 AV,01/03/2018,20417,22357,1st Ave,2,40.730975,-73.981681,42774
3,1 AV,01/04/2018,12010,13472,1st Ave,3,40.730975,-73.981681,25482
4,1 AV,01/05/2018,18891,21966,1st Ave,4,40.730975,-73.981681,40857


In [122]:
df.to_pickle('df_final_update.pickle')

In [123]:
type(df_sta.lat[0])

numpy.float64

In [124]:
byMean = df.groupby('STATION').mean()
byMean.head()

,ENTRIES,EXITS,weekday,lat,long,volume
STATION,,,,,,
1 AV,16640.285714,17975.576923,3.0,40.730975,-73.981681,34615.862637
103 ST,24898.337912,15611.291209,3.0,40.799446,-73.968379,40509.629121
103 ST-CORONA,16990.513736,10904.483516,3.0,40.799446,-73.968379,27894.997253
104 ST,2476.923077,701.524725,3.0,40.681711,-73.837683,3178.447802
110 ST,9581.337912,6975.239011,3.0,40.795020,-73.944250,16556.576923


In [125]:
ct = pd.read_pickle('census_by_latlong.pickle')
ct.head()

,CensusTract,TotalPop,Women,Income,Poverty,Transit,Employed,center_loc
0,36061000100,0,0,NaN,NaN,NaN,0,"[-74.0457717317, 40.6903306467]"
1,36061000201,2791,1490,20521.0,48.6,45.4,1105,"[-73.9857826696, 40.7119512739]"
2,36061000202,7768,4454,29684.0,24.6,41.5,2667,"[-73.9820376761, 40.7056795757]"
3,36061000500,0,0,NaN,NaN,NaN,0,"[-74.0095085304, 40.6855017439]"
4,36061000600,12554,6588,19863.0,44.7,37.0,4028,"[-73.9899595444, 40.7111190321]"


In [126]:
def FindCensusTract(df1,df2):
    df1['CensusTract'] = np.nan
    
    for i in range(len(df1)):
        xy = np.array([df1.long[i],df1.lat[i]])
        dif = np.array([np.square(df2.iloc[k].center_loc[0]-xy[0])+np.square(df2.iloc[k].center_loc[1]-xy[1]) 
                        for k in range(len(df2))])
        ind = dif.argmin()
        df1.at[i,'CensusTract'] = df2['CensusTract'][ind]

    return(df1)

In [127]:
byMean = byMean.reset_index()
len(byMean)

380

In [128]:
temp = FindCensusTract(byMean,ct)

In [129]:
temp['CensusTract'] = temp.CensusTract.apply(lambda x: int(x))

In [132]:
temp.head()

,STATION,ENTRIES,EXITS,weekday,lat,long,volume,CensusTract
0,1 AV,16640.285714,17975.576923,3.0,40.730975,-73.981681,34615.862637,36061003400
1,103 ST,24898.337912,15611.291209,3.0,40.799446,-73.968379,40509.629121,36061019100
2,103 ST-CORONA,16990.513736,10904.483516,3.0,40.799446,-73.968379,27894.997253,36061019100
3,104 ST,2476.923077,701.524725,3.0,40.681711,-73.837683,3178.447802,36061004400
4,110 ST,9581.337912,6975.239011,3.0,40.795020,-73.944250,16556.576923,36061017200


In [134]:
df_new = temp.merge(ct, on='CensusTract')
df_new = df_new.set_index('STATION')
df_new.head()

,ENTRIES,EXITS,weekday,lat,long,volume,CensusTract,TotalPop,Women,Income,Poverty,Transit,Employed,center_loc
STATION,,,,,,,,,,,,,,
1 AV,16640.285714,17975.576923,3.0,40.730975,-73.981681,34615.862637,36061003400,6476,3313,67166.0,24.1,55.5,4480,"[-73.9815026357, 40.7292172249]"
103 ST,24898.337912,15611.291209,3.0,40.799446,-73.968379,40509.629121,36061019100,9545,5438,83188.0,10.2,70.0,6057,"[-73.9706439126, 40.8010485594]"
103 ST-CORONA,16990.513736,10904.483516,3.0,40.799446,-73.968379,27894.997253,36061019100,9545,5438,83188.0,10.2,70.0,6057,"[-73.9706439126, 40.8010485594]"
104 ST,2476.923077,701.524725,3.0,40.681711,-73.837683,3178.447802,36061004400,16200,9307,93010.0,13.1,58.1,10148,"[-73.9623232245, 40.7241237808]"
111 ST,14248.604396,7461.909341,3.0,40.684331,-73.832163,21710.513736,36061004400,16200,9307,93010.0,13.1,58.1,10148,"[-73.9623232245, 40.7241237808]"


In [6]:
# from scipy import stats
# import numpy as np
# df = df[(np.abs(stats.zscore(df)) < 3).all(axis=1)]

In [135]:
df_new.sort_values('volume',ascending=False).head(10)

,ENTRIES,EXITS,weekday,lat,long,volume,CensusTract,TotalPop,Women,Income,Poverty,Transit,Employed,center_loc
STATION,,,,,,,,,,,,,,
34 ST-PENN STA,127057.491758,111318.032967,3.0,40.750373,-73.991057,238375.524725,36061010100,1259,501,159821.0,11.4,41.1,932,"[-73.9907618992, 40.7498218385]"
GRD CNTRL-42 ST,111173.670330,105393.906593,3.0,40.751807,-73.976713,216567.576923,36061008000,5045,2740,110242.0,10.5,32.2,3307,"[-73.9776066526, 40.749652967]"
34 ST-HERALD SQ,87010.483516,89816.538462,3.0,40.750373,-73.991057,176827.021978,36061010100,1259,501,159821.0,11.4,41.1,932,"[-73.9907618992, 40.7498218385]"
14 ST-UNION SQ,78714.134615,75190.552198,3.0,40.808719,-73.907657,153904.686813,36061019200,4066,2424,16206.0,56.6,77.2,1012,"[-73.9273780701, 40.7973262874]"
23 ST,84724.524725,66607.587912,3.0,40.884667,-73.900870,151332.112637,36061030900,9398,5145,27513.0,30.4,60.0,4022,"[-73.9117451419, 40.8746105507]"
TIMES SQ-42 ST,72945.464286,74993.425824,3.0,40.754612,-73.986768,147938.890110,36061011300,126,30,82361.0,19.0,44.4,108,"[-73.9875488222, 40.7548351913]"
FULTON ST,72536.118132,61153.041209,3.0,40.677316,-73.983110,133689.159341,36061000900,1626,680,147500.0,6.0,51.9,1120,"[-74.0014427365, 40.6947309138]"
42 ST-PORT AUTH,69825.346154,59053.791209,3.0,40.757308,-73.989735,128879.137363,36061012100,8192,3622,62301.0,9.9,41.8,5750,"[-73.9916459695, 40.7598094212]"
86 ST,68711.368132,56119.618132,3.0,40.788644,-73.976218,124830.986264,36061017500,10840,5631,137344.0,3.5,71.3,5915,"[-73.9779910059, 40.7905780434]"


In [137]:
byMean = df_new.drop(['weekday','center_loc'],axis=1)
byMean.head()

,ENTRIES,EXITS,lat,long,volume,CensusTract,TotalPop,Women,Income,Poverty,Transit,Employed
STATION,,,,,,,,,,,,
1 AV,16640.285714,17975.576923,40.730975,-73.981681,34615.862637,36061003400,6476,3313,67166.0,24.1,55.5,4480
103 ST,24898.337912,15611.291209,40.799446,-73.968379,40509.629121,36061019100,9545,5438,83188.0,10.2,70.0,6057
103 ST-CORONA,16990.513736,10904.483516,40.799446,-73.968379,27894.997253,36061019100,9545,5438,83188.0,10.2,70.0,6057
104 ST,2476.923077,701.524725,40.681711,-73.837683,3178.447802,36061004400,16200,9307,93010.0,13.1,58.1,10148
111 ST,14248.604396,7461.909341,40.684331,-73.832163,21710.513736,36061004400,16200,9307,93010.0,13.1,58.1,10148


In [144]:
byMeanWeekend = df[df.weekday>4].groupby('STATION').mean()

In [148]:
byMean['perc_weekend'] = 100*byMeanWeekend.volume/byMean.volume
byMean.head()

,ENTRIES,EXITS,lat,long,volume,CensusTract,TotalPop,Women,Income,Poverty,Transit,Employed,perc_weekend
STATION,,,,,,,,,,,,,
1 AV,16640.285714,17975.576923,40.730975,-73.981681,34615.862637,36061003400,6476,3313,67166.0,24.1,55.5,4480,70.649830
103 ST,24898.337912,15611.291209,40.799446,-73.968379,40509.629121,36061019100,9545,5438,83188.0,10.2,70.0,6057,62.904773
103 ST-CORONA,16990.513736,10904.483516,40.799446,-73.968379,27894.997253,36061019100,9545,5438,83188.0,10.2,70.0,6057,71.151096
104 ST,2476.923077,701.524725,40.681711,-73.837683,3178.447802,36061004400,16200,9307,93010.0,13.1,58.1,10148,44.795390
111 ST,14248.604396,7461.909341,40.684331,-73.832163,21710.513736,36061004400,16200,9307,93010.0,13.1,58.1,10148,65.702835


In [149]:
byMean.to_pickle('byMean.pickle')